In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')
#if you do not have access to the twenty qubit machine, use this publically available 14 qubit machine instead
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_20_tokyo') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>,
 <IBMQBackend('ibmq_almaden') from IBMQ(hub='ibm-q-ncsu', group='nc-state

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the 20qubit boeblingen instead
device = provider.get_backend('ibmq_boeblingen')
backend_monitor(device)
device

ibmq_boeblingen
Configuration
-------------
    n_qubits: 20
    operational: True
    status_msg: active
    pending_jobs: 88
    backend_version: 1.0.3
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    credits_required: True
    online_date: 2019-07-03T04:00:00+00:00
    quantum_volume: 16
    open_pulse: True
    description: 20 qubit device Boeblingen
    allow_q_object: True
    meas_levels: [1, 2]
    url: None
    hamiltonian: {'description': 'Qubits are modelled as a two level system. Qubits are coupled through resonator buses. The provided Hamiltonian has been projected into the zero excitation subspace of the resonator buses leading to an effective qubit-qubit flip-flop interaction. The qubit resonance frequencies in the Hamiltonian are the cavity dressed frequencies and not exactly what is returned by the backend defaults, which also includes the dressing due to the qubit-qubit interactions.\nWarning: Currently not all system Hamiltoni

<IBMQBackend('ibmq_boeblingen') from IBMQ(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')>

In [5]:
transQcList = []

pi = math.pi

In [6]:
import numpy as np
def input_state(circ, nqubits, value):
    for j in range(nqubits):
        circ.h(j)
        circ.u1(-value * math.pi/float(2**(j)), j)
def qft(circ, q, n):
    """n-qubit QFT on q in circ."""
    for j in range(n):
        for k in range(j):
            circ.cu1(np.pi / float(2**(j - k)), q[j], q[k])
        circ.h(q[j])

In [7]:
def qpeExperiment(value):
    q = QuantumRegister(5)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    
    circuit.x(q[4])
    input_state(circuit, 4, value)
    
    
    qft(circuit, q, 4)

    for k in range(4):
        circuit.measure(q[k],c[k])
    
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    transQcList.append(trans_qc)
    print("QPE circuit size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())

    
    q = QuantumRegister(5)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    input_state(circuit, 4, value)
        

    qft(circuit, q, 4)
    qubitList = [q[3]]
    
    assertion_value = int("{0:04b}".format(value)[0])
    print("{0:04b}".format(value))
    print(assertion_value)
    classical_assertion(circuit, qubitList, assertion_value)

    for k in range(4):
        circuit.measure(q[k],c[k])

        
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    transQcList.append(trans_qc)
    print("QPE circuit with assertion size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
    circuit.draw()

In [9]:
qpeExperiment(0)
qpeExperiment(1)
qpeExperiment(8)
qpeExperiment(3)
qpeExperiment(12)
qpeExperiment(7)
qpeExperiment(14)
qpeExperiment(15)

QPE circuit size = 49 depth =  36 width =  24 
 OPs = OrderedDict([('cx', 24), ('u1', 14), ('u2', 6), ('measure', 4), ('u3', 1), ('barrier', 1)]) subcircuits = 17
0000
0
QPE circuit with assertion size = 63 depth =  45 width =  25 
 OPs = OrderedDict([('cx', 30), ('u3', 10), ('u2', 9), ('u1', 9), ('measure', 5), ('barrier', 1)]) subcircuits = 16
QPE circuit size = 50 depth =  35 width =  24 
 OPs = OrderedDict([('cx', 24), ('u1', 14), ('u2', 6), ('measure', 4), ('u3', 2), ('barrier', 1)]) subcircuits = 17
0001
0
QPE circuit with assertion size = 64 depth =  44 width =  25 
 OPs = OrderedDict([('cx', 30), ('u3', 13), ('u1', 10), ('u2', 6), ('measure', 5), ('barrier', 1)]) subcircuits = 16
QPE circuit size = 51 depth =  35 width =  24 
 OPs = OrderedDict([('cx', 22), ('u1', 10), ('u2', 9), ('u3', 6), ('measure', 4), ('barrier', 1)]) subcircuits = 17
1000
1
QPE circuit with assertion size = 58 depth =  41 width =  25 
 OPs = OrderedDict([('cx', 29), ('u1', 10), ('u2', 9), ('u3', 5), ('mea

In [10]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of output state |0000> without assertion

In [12]:
calcSuccessrate(res.get_counts(0), "0000", 0)

total_count =  8192 success_count =  1994 success_rate =  24.3408203125 %


Success rate of output state |0000> with assertion

In [14]:
calcSuccessrate(res.get_counts(1), "0 0000", 1)

total_count =  4598 success_count =  647 success_rate =  14.071335363201392 %


Success rate of output state |0001> without assertion

In [15]:
calcSuccessrate(res.get_counts(2), "0001", 0)

total_count =  8192 success_count =  2668 success_rate =  32.568359375 %


Success rate of output state |0001> with assertion

In [17]:
calcSuccessrate(res.get_counts(3), "0 0001", 1)

total_count =  4280 success_count =  2138 success_rate =  49.953271028037385 %


Success rate of output state |1000> without assertion

In [18]:
calcSuccessrate(res.get_counts(4), "1000", 0)

total_count =  8192 success_count =  1796 success_rate =  21.923828125 %


Success rate of output state |1000> with assertion

In [19]:
calcSuccessrate(res.get_counts(5), "0 1000", 1)

total_count =  3965 success_count =  859 success_rate =  21.664564943253467 %


Success rate of output state |0011> without assertion

In [20]:
calcSuccessrate(res.get_counts(6), "0011", 0)

total_count =  8192 success_count =  4083 success_rate =  49.84130859375 %


Success rate of output state |0011> with assertion

In [21]:
calcSuccessrate(res.get_counts(7), "0 0011", 1)

total_count =  5705 success_count =  3438 success_rate =  60.26292725679229 %


Success rate of output state |1100> without assertion

In [22]:
calcSuccessrate(res.get_counts(8), "1100", 0)

total_count =  8192 success_count =  1633 success_rate =  19.93408203125 %


Success rate of output state |1100> with assertion

In [23]:
calcSuccessrate(res.get_counts(9), "0 1100", 1)

total_count =  3503 success_count =  673 success_rate =  19.212103910933486 %


Success rate of output state |0111> without assertion

In [24]:
calcSuccessrate(res.get_counts(10), "0111", 0)

total_count =  8192 success_count =  2966 success_rate =  36.2060546875 %


Success rate of output state |0111> with assertion

In [25]:
calcSuccessrate(res.get_counts(11), "0 0111", 1)

total_count =  3836 success_count =  1689 success_rate =  44.03023983315954 %


Success rate of output state |1110> without assertion

In [26]:
calcSuccessrate(res.get_counts(12), "1110", 0)

total_count =  8192 success_count =  2400 success_rate =  29.296875 %


Success rate of output state |1110> with assertion

In [27]:
calcSuccessrate(res.get_counts(13), "0 1110", 1)

total_count =  3056 success_count =  1190 success_rate =  38.93979057591623 %


Success rate of output state |1111> without assertion

In [28]:
calcSuccessrate(res.get_counts(14), "1111", 0)

total_count =  8192 success_count =  1897 success_rate =  23.15673828125 %


Success rate of output state |1111> with assertion

In [29]:
calcSuccessrate(res.get_counts(15), "0 1111", 1)

total_count =  3676 success_count =  661 success_rate =  17.981501632208925 %
